Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 4

## Assignment

- [ ] Watch Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.
- [ ] Do train/validate/test split with the Tanzania Waterpumps data.
- [ ] Do one-hot encoding. (Remember it may not work with high cardinality categoricals.)
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your coefficients.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

> [Do Not Copy-Paste.](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit) You must type each of these exercises in, manually. If you copy and paste, you might as well not even do them. The point of these exercises is to train your hands, your brain, and your mind in how to read, write, and see code. If you copy-paste, you are cheating yourself out of the effectiveness of the lessons.


## Stretch Goals

### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- [ ] Make exploratory visualizations.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from the previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```

#### Pipelines

[Scikit-Learn User Guide](https://scikit-learn.org/stable/modules/compose.html) explains why pipelines are useful, and demonstrates how to use them:

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:
> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

### Reading
- [ ] [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)
- [ ] [Always start with a stupid model, no exceptions](https://blog.insightdatascience.com/always-start-with-a-stupid-model-no-exceptions-3a22314b9aaa)
- [ ] [Statistical Modeling: The Two Cultures](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).



In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module4')

In [2]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
import pandas as pd

train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
xtrain, xval, ytrain, yval= train_test_split(train_features,train_labels)
xtrain.shape, xval.shape, ytrain.shape, yval.shape

((44550, 40), (14850, 40), (44550, 2), (14850, 2))

In [43]:
def cleanUp(df):
    df=df.drop(columns=['id','num_private','amount_tsh'])
    df.date_recorded=pd.to_datetime(df.date_recorded)
    df.longitude=df.longitude.replace({0:35.15315303394668})
    df.latitude=df.latitude.replace({0:-5.699241279889469})
    df.construction_year=df.construction_year.replace({0:2010})
    df.population=df.population.replace({0:281.60745437348936})
    df.region_code=df.region_code.astype(str)
    df.district_code=df.district_code.astype(str)
    features=['basin','district_code', 'extraction_type', 'extraction_type_class', 'extraction_type_group', 'management', 'management_group','payment','payment_type','permit',
 'public_meeting',
 'quality_group',
 'quantity',
 'quantity_group',
 'recorded_by',
 'region',
 'region_code',
 'scheme_management',
 'source',
 'source_class',
 'source_type',
 'water_quality',
 'waterpoint_type',
 'waterpoint_type_group',
 'gps_height',
 'longitude',
 'latitude',
 'population',
 'construction_year']
    df=df[features]
    return df
    
def cleanuptarget(y):
    y=y.status_group
    return y

In [44]:
xtrain=cleanUp(xtrain)
xval=cleanUp(xval)
xtest=cleanUp(test_features)
ytrain=cleanuptarget(ytrain)
yval=cleanuptarget(yval)

KeyError: "['id' 'num_private' 'amount_tsh'] not found in axis"

In [8]:
#xtrain=xtrain.drop(columns=['id','num_private', "amount_tsh"])
yval

54793             non functional
43588             non functional
35444                 functional
1839                  functional
44185                 functional
58109                 functional
56785                 functional
37747             non functional
41591             non functional
24776                 functional
14735                 functional
26305             non functional
48694                 functional
13083             non functional
3765                  functional
41562                 functional
32723                 functional
46789                 functional
7049                  functional
4704                  functional
15443                 functional
58762                 functional
39027             non functional
35410                 functional
1646                  functional
12129                 functional
1773                  functional
43670                 functional
26421             non functional
46923             non functional
          

In [9]:
#xval=xval.drop(columns=['id','num_private','amount_tsh'])

In [10]:
#xtrain.date_recorded=pd.to_datetime(xtrain.date_recorded)

In [11]:
#xval.date_recorded=pd.to_datetime(xval.date_recorded)

In [12]:
#xtrain.population.replace({0:np.nan}).mean()

In [13]:
import numpy as np

In [14]:
# xtrain.longitude=xtrain.longitude.replace({0:np.nan})
# xtrain.longitude=xtrain.longitude.replace({np.nan:35.15315303394668})




In [15]:
# xtrain.latitude=xtrain.latitude.replace({0:np.nan})
# xtrain.latitude.mean()
# xtrain.latitude=xtrain.latitude.replace({np.nan:-5.699241279889469})
# xtrain.latitude


In [16]:
# xtrain.construction_year=xtrain.construction_year.replace({0:np.NaN})
# xtrain.construction_year=xtrain.construction_year.replace({np.nan:2010})

# xtrain


In [17]:
# #Determine Majoirty Class
# yval.status_group.value_counts()

In [18]:

ytrain.value_counts(normalize=True) #So baseline model would get 54 percent correct.

functional                 0.543996
non functional             0.383591
functional needs repair    0.072413
Name: status_group, dtype: float64

In [19]:
# totaltrain=xtrain.copy()
# totaltrain['target']=ytrain.status_group
# totaltrain.groupby('region').target.describe()

In [21]:
catagorical_features=xtrain.select_dtypes(exclude="number").columns
numeric_features=xtrain.select_dtypes('number').columns

In [22]:
numeric_features


Index(['gps_height', 'longitude', 'latitude', 'population',
       'construction_year'],
      dtype='object')

In [23]:
catagorical_features

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name',
       'permit', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [24]:
# xtrain['functional_number']=ytrain.status_group.copy().replace({'non functional':0,
#                                                         'functional needs repair':1,
#                                                         'functional':2})
# xtrain.head()

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
low_cardinality_cat_feat=[]
for col in sorted(catagorical_features):
    if xtrain[col].nunique()<=30:
#         sns.catplot(x=col, y='functional_number', data=xtrain, kind='bar', color='blue')
#         plt.show()
        low_cardinality_cat_feat.append(col)

In [27]:
low_cardinality_cat_feat

['basin',
 'district_code',
 'extraction_type',
 'extraction_type_class',
 'extraction_type_group',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'permit',
 'public_meeting',
 'quality_group',
 'quantity',
 'quantity_group',
 'recorded_by',
 'region',
 'region_code',
 'scheme_management',
 'source',
 'source_class',
 'source_type',
 'water_quality',
 'waterpoint_type',
 'waterpoint_type_group']

In [28]:
features=list(low_cardinality_cat_feat)+list(numeric_features)

In [29]:
features

['basin',
 'district_code',
 'extraction_type',
 'extraction_type_class',
 'extraction_type_group',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'permit',
 'public_meeting',
 'quality_group',
 'quantity',
 'quantity_group',
 'recorded_by',
 'region',
 'region_code',
 'scheme_management',
 'source',
 'source_class',
 'source_type',
 'water_quality',
 'waterpoint_type',
 'waterpoint_type_group',
 'gps_height',
 'longitude',
 'latitude',
 'population',
 'construction_year']

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
from sklearn.linear_model import LogisticRegressionCV

model=LogisticRegressionCV(n_jobs=-1, cv=5, multi_class='auto')

model.fit(xtrain[numeric_features],ytrain)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                     random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [32]:
from sklearn.metrics import accuracy_score
ypred=model.predict(xval[numeric_features])
accuracy_score(yval, ypred)

0.5396632996632996

In [33]:
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler

In [34]:
xtrain_subset=xtrain[features]
xval_subset=xval[features]

encoder=ce.OneHotEncoder()
xtrain_encoded=encoder.fit_transform(xtrain_subset)
xval_encoded=encoder.transform(xval_subset)

In [35]:
scaler=MinMaxScaler()
xtrain_scaled=scaler.fit_transform(xtrain_encoded)
xval_scaled=scaler.transform(xval_encoded)


In [36]:
model=LogisticRegressionCV(n_jobs=-1, multi_class='auto', cv=5, max_iter=1000)
model.fit(xtrain_scaled, ytrain)
print("Validation Accuracy", model.score(xval_scaled,yval))

Validation Accuracy 0.7408754208754209


In [40]:
from sklearn.feature_selection import f_regression, SelectKBest

In [41]:
for k in range (1, len(features)+1):
    print(f'{k} features')
    
    selector=SelectKBest(score_func=f_regression, k=k)
    x_train_selected=selector.fit_transform(xtrain_scaled,ytrain)
    x_val_selected=selector.transform(xval_scaled)
    
    model=LogisticRegressionCV(n_jobs=-1, multiclass='auto', cv=5, max_iter=1000)
    model.fit(x_train_selected, ytrain)
    print("Validation accuracy", model.score(x_val_selected, yval))
    

1 features


TypeError: unsupported operand type(s) for /: 'str' and 'int'